In [7]:
import sys
sys.path.append('../../src/')
import sympy
from sympy import symbols
from sympy.physics.mechanics import dynamicsymbols,ReferenceFrame
import numpy as np
from dsm import getComponents
from exporter import ExportFunPy

t=symbols('t')
#Parametros inerciales del modelo
[m,Icm]=symbols('m,Icm')
#Parametros geometricos del modelo
[x1,y1,x2,y2,x3,y3,x4,y4,beta1,beta2,beta3,beta4]=symbols('x1,y1,x2,y2,x3,y3,x4,y4,beta1,beta2,beta3,beta4')
#Parametros dinámicos del modelo
[theta,x,y]=dynamicsymbols('theta,x,y')

#Definir marcos de referencia de importancia
N=ReferenceFrame('N')
A=N.orientnew('A','Axis',(theta,N.z))
B1=A.orientnew('B1','Axis',(beta1,N.z))
B2=A.orientnew('B2','Axis',(-beta2,N.z))
B3=A.orientnew('B3','Axis',(sympy.pi-beta3,N.z))
B4=A.orientnew('B4','Axis',(sympy.pi+beta4,N.z))

#Definir las posiciones en que se aplica cada fuerza
# y la magnitud de cada fuerza en funcion de escalares u_i
r1=x1*A.x+y1*A.y
r2=x2*A.x+y2*A.y
r3=x3*A.x+y3*A.y
r4=x4*A.x+y4*A.y
[u1,u2,u3,u4]=symbols('u1,u2,u3,u4')
F1=-u1*B1.y
F2=-u2*B2.y
F3=u3*B3.y
F4=u4*B4.y

a_cm=x.diff(t,t)*N.x+y.diff(t,t)*N.y
# Las ecuaciones son más sencillas si trabajamos la aceleración expresada en A
# e.g. a_cm=x.diff(t,t)*A.x+y.diff(t,t)*A.y

#Ecuaciones newton-euler
eqForces=F1+F2+F3+F4-m*a_cm
eqMoments=r1.cross(F1)+r2.cross(F2)+r3.cross(F3)+r4.cross(F4)-Icm*theta.diff(t,t)*N.z
eqs=getComponents(eqForces,A)[0:2]+[getComponents(eqMoments,A)[2]]

sln=sympy.solve(eqs,[x.diff(t,t),y.diff(t,t),theta.diff(t,t)],simplify=True)

In [8]:
sln[theta.diff(t,t)]

(-u1*x1*cos(beta1) - u1*y1*sin(beta1) - u2*x2*cos(beta2) + u2*y2*sin(beta2) - u3*x3*cos(beta3) + u3*y3*sin(beta3) - u4*x4*cos(beta4) - u4*y4*sin(beta4))/Icm

In [9]:
sln[y.diff(t,t)]

-(u1*cos(beta1 + theta(t)) + u2*cos(beta2 - theta(t)) + u3*cos(beta3 - theta(t)) + u4*cos(beta4 + theta(t)))/m

In [10]:
sln[x.diff(t,t)]

(u1*sin(beta1 + theta(t)) - u2*sin(beta2 - theta(t)) - u3*sin(beta3 - theta(t)) + u4*sin(beta4 + theta(t)))/m

In [11]:
params={beta1:np.deg2rad(45),beta2:np.deg2rad(45),beta3:np.deg2rad(45),beta4:np.deg2rad(45),
       x1:0.20,x2:-0.20,x3:0.20,x4:-0.20,y1:0.20,y2:0.20,y3:-0.20,y4:-0.20,
       m:1,Icm:1*(0.2**2*2/12)}

'''
Parametros que nunca dejarían rotar el ROV
params={beta1:np.deg2rad(-45),beta2:np.deg2rad(-45),beta3:np.deg2rad(-45),beta4:np.deg2rad(-45),
       x1:0.20,x2:-0.20,x3:0.20,x4:-0.20,y1:0.20,y2:0.20,y3:-0.20,y4:-0.20,
       m:1,Icm:1*(0.2**2*2/12)}
'''

thetaddot_sln=sln[theta.diff(t,t)].subs(params)
xddot_sln=sln[x.diff(t,t)].subs(params).simplify()
yddot_sln=sln[y.diff(t,t)].subs(params).simplify()

In [12]:
thetaddot_sln.simplify()

-42.4264068711929*u1 + 42.4264068711929*u2 - 42.4264068711929*u3 + 42.4264068711929*u4

In [13]:
vars=[theta,x,y]
varsdot=[var_i.diff(t) for var_i in vars]
qvars=vars+varsdot
uvars=[u1,u2,u3,u4]
thetaddot_fun=sympy.lambdify(vars+varsdot+uvars,thetaddot_sln)
xddot_fun=sympy.lambdify(vars+varsdot+uvars,xddot_sln)
yddot_fun=sympy.lambdify(vars+varsdot+uvars,yddot_sln)

def dq_dt(q,t=None,u=None):
    if u==None:
        u=np.zeros(4)
    qdot=np.zeros(6)
    qdot[0]=q[3]
    qdot[1]=q[4]
    qdot[2]=q[5]
    qdot[3]=thetaddot_fun(*q,*u)
    qdot[4]=xddot_fun(*q,*u)
    qdot[5]=yddot_fun(*q,*u)
    return qdot

# Probar
#uvalues=[1,1,1,1]
#qvalues=[0,0,0,0,0,0]
#yddot_fun(*(qvalues+uvalues))



-2.828427124746191

In [16]:
ExportFunPy(dq_dt)
ExportFunPy(thetaddot_fun,rename='thetaddot_fun')
ExportFunPy(xddot_fun,rename='xddot_fun')
ExportFunPy(yddot_fun,rename='yddot_fun')
